# 1. 개인 구글 드라이브와 colab 연동

In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


# 2. 라이브러리 설치

In [2]:
!pip install sklearn-crfsuite

# 3. 데이터 전처리

<pre>
1. 데이터셋에서 json 파일을 불러온다.
2. 데이터셋의 json 파일에서 카테고리별로 named_entity 항목의 content를 가져온다.
3. 카테고리별 합의 비율별에 랜덤하게 문장 1만개를 뽑는다.
4. content의 띄어쓰기 정보를 이용해 각 content에 대응하는 띄어쓰기 레이블을 가져온다.

결과물:
train_datas
    ㄴ category
        ㄴ sentences
</pre>

In [1]:
import os
import json
import sklearn_crfsuite
from sklearn_crfsuite import metrics

RAW_TRAIN_PATH = './rawDataset'
TRAIN_PATH = './dataset'

In [54]:
def preprocess_data(source_path, new_path):
    categories = os.listdir(path=source_path) # 건강, 경제, 교육, ...
    print(categories)
    
    for category in categories:
        for filename in os.listdir(os.path.join(source_path, category)):
            result = []
            
            with open(os.path.join(source_path, category, filename), 'r', encoding='UTF-8') as source_file,\
                open(os.path.join(new_path, category + '_spacing_data.txt'), 'w', encoding='UTF-8-sig') as new_file:
                
                # Read
                json_data = json.load(source_file)

                for named_entity in json_data['named_entity']:
                    for item in named_entity['content']:
                        # Preprocess
                        new_sample = item['sentence'][0] + ' '
                        label = 'B '
                        
                        it = iter(item['sentence'])
                        for idx, c in enumerate(item['sentence'][1:], 1):
                            if c == ' ':
                                continue
                                
                            new_sample += (c + ' ')
                            if item['sentence'][idx - 1] == ' ':
                                label += 'B '
                            else:
                                label += 'I '

                        result = new_sample + '\t' + label + '\n'
                            
                        # Write
                        new_file.write(result)

In [56]:
# Preprocess data
preprocess_data(RAW_TRAIN_PATH, TRAIN_PATH)
                    
print("imported sentences of train_datas completely")

['IT_과학', '건강', '경제', '교육', '국제', '라이프스타일', '문화', '사건사고', '사회일반', '산업', '스포츠', '여성복지', '여행레저', '연예', '정치', '지역', '취미']
imported sentences of train_datas completely


In [2]:
def get_dataset_num(path, target_num):
    file_lines = {}
    total_lines = 0
    
    for filename in os.listdir(path):
        with open(os.path.join(path, filename), 'r',  encoding='UTF-8') as file:
            lines = file.readlines()
            file_lines[filename] = len(lines)
            total_lines += len(lines)
    
    target_lines = {}
    total_target = 0
    
    for filename in os.listdir(path):
        num_lines = file_lines[filename]
        target_lines[filename] = round(num_lines * target_num / total_lines)    
        total_target += target_lines[filename]

    # Fine tuning
    target_lines[os.listdir(path)[0]] += (target_num - total_target)
    
    for filename in os.listdir(path):
        num_lines = file_lines[filename]
        pi = file_lines[filename] / total_lines
    
    total_target = sum([target_lines[filename] for filename in os.listdir(path)])
    return target_lines

In [3]:
# Set dataset num
dataset_num = get_dataset_num(TRAIN_PATH, 10000)
print(dataset_num)

print(f"total: {sum([dataset_num[filename] for filename in dataset_num])}")

{'IT_과학_spacing_data.txt': 952, '건강_spacing_data.txt': 1048, '경제_spacing_data.txt': 817, '교육_spacing_data.txt': 569, '국제_spacing_data.txt': 655, '라이프스타일_spacing_data.txt': 210, '문화_spacing_data.txt': 882, '사건사고_spacing_data.txt': 319, '사회일발_spacing_data.txt': 132, '산업_spacing_data.txt': 377, '스포츠_spacing_data.txt': 902, '여성복지_spacing_data.txt': 685, '여행레저_spacing_data.txt': 496, '연예_spacing_data.txt': 635, '정치_spacing_data.txt': 308, '지역_spacing_data.txt': 434, '취미_spacing_data.txt': 579}
total: 10000


<pre>
1. 문장과 라벨을 튜플형태로 datas 리스트에 넣는다
    datas = [ ( ["나", "는", "사", "과", "가", "좋", "아"], ["B", "I", "B", "I", "I", "B", "I"] ), ( ... ), ... ]

2. 전체 데이터를 분류 비율에 맞추어, 테스트-검증 9:1 비율에 맞추어 학습, 평가 데이터로 나누기
  train_datas 리스트와 test_datas 리스트에 나누어 저장
</pre>

In [4]:
import random

train_datas = []
test_datas = []

for filename in os.listdir(TRAIN_PATH):
    # 파일을 읽고 lines에 읽은 데이터를 저장
    with open(os.path.join(TRAIN_PATH, filename), "r", encoding="utf8") as file:
        lines = file.readlines()
    
    # 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
    datas = []

    # 데이터 개수 10000개로 맞추기
    lines = random.sample(lines, dataset_num[filename])
    for line in lines:
        pieces = line.strip().split('\t')
        eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
        datas.append((eumjeol_sequence, label))
    
    number_of_train_datas = round(len(datas)*0.9)
    
    train_datas += datas[:number_of_train_datas]
    test_datas += datas[number_of_train_datas:]

print("train_datas 개수 : " + str(len(train_datas)))
print("test_datas 개수 : " + str(len(test_datas)))

train_datas 개수 : 9000
test_datas 개수 : 1000


<pre>
1. 문장의 각 음절을 crf 모델의 입력으로 사용 할 수 있도록 자질화 </h2>
  "BOS" : 시작 음절인지 여부
  "EOS" : 마지막 음절인지 여부
  "WORD" : 기준 음절
  "IS_DIGIT" : 기준 음절이 숫자인지 여부
  "-1_WORD" : 기준 음절의 왼쪽 첫번째 음절
  "-2_WORD" : 기준 음절의 왼쪽 두번째 음절
  "+1_WORD" : 기준 음절의 오른쪽 첫번째 음절
  "+2_WORD" : 기준 음절의 오른쪽 두번째 음절

    예시) ["나", "는", "사", "과", "가", "좋", "아"]
         -> [
    { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
    { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
    { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" },
    ... ]

2. 자질화한 데이터와 해당 데이터의 라벨을 분리하여 각 리스트에 저장
  학습 데이터 -> train_x(자질화한 데이터), train_y(각 데이터의 정답 라벨)에 저장
  평가 데이터 -> test_x(자질화한 데이터), test_y(각 데이터의 정답 라벨)에 저장
  
    예시)
    train_x -> [
        [ { "BOS":True, "EOS":False, "WORD":"나", "IS_DIGIT":False, "+1_WORD":"는", "+2_WORD":"사" },
        { "BOS":False, "EOS":False, "WORD":"는", "IS_DIGIT":False, "-1_WORD":"나", "+1_WORD":"사", "+2_WORD":"과" },
        { "BOS":False, "EOS":False, "WORD":"사", "IS_DIGIT":False, "-2_WORD":"나", "-1_WORD":"는", "+1_WORD":"과", "+2_WORD":"가" },
        ... ],
        ...
    ]
    
    train_y -> [
        [ "B", "I", "B", "I", "I", "B", "I" ],
        ...
    ]

</pre>

In [5]:
def sent2feature(eumjeol_sequence):
    features = []
    sequence_length = len(eumjeol_sequence)
    for index, eumjeol in enumerate(eumjeol_sequence):
        feature = {
            'BOS': False,
            'EOS': False,
            'WORD': eumjeol,
            'IS_DIGIT':eumjeol.isdigit()
        }

        if index == 0:
            feature["BOS"] = True
        elif index == sequence_length - 1:
            feature['EOS'] = True

        if index - 1 >= 0:
            feature['-1_WORD'] = eumjeol_sequence[index - 1]
        if index - 2 >= 0:
            feature['-2_WORD'] = eumjeol_sequence[index - 2]

        if index + 1 <= sequence_length - 1:
            feature['+1_WORD'] = eumjeol_sequence[index + 1]
        if index + 2 <= sequence_length - 2:
            feature['+2_WORD'] = eumjeol_sequence[index + 2]

        features.append(feature)
    return features


train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

In [6]:
print(f"{train_x[0][9]['WORD']} -> {train_x[0][9]}")

프 -> {'BOS': False, 'EOS': False, 'WORD': '프', 'IS_DIGIT': False, '-1_WORD': '망', '-2_WORD': '경', '+1_WORD': '로', '+2_WORD': '세'}


# 3. train_x, train_y를 이용하여 crf 모델 학습

In [7]:
#crf = sklearn_crfsuite.CRF(
#        algorithm='lbfgs',
#        c1=0.1,
#        c2=0.1,
#        max_iterations=100,
#        all_possible_transitions=True
#    )

crf = sklearn_crfsuite.CRF()
try:
    crf.fit(train_x, train_y)
except AttributeError:
    pass

<pre>
1. 학습한 모델을 test_x 데이터를 사용하여 평가
2. 성능 측정
    metrics.flat_accuracy_score(x, y) 함수를 이용하여 성능 측정
  
3. 모델의 출력 값과 정답 값을 이용하여 음절만으로 구성된 완전한 문장으로 변형
    예시)
    정답 문장 : 하지만 이 전쟁은 죽음을 통해 인류를 통합시켰다.
    출력 문장 : 하지만이 전쟁은 죽음을 통해 인류를 통합시켰다.
    ...
</pre>


In [8]:
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]
      predict_labels = predict[index_1]

      correct_sentence, predict_sentence = "", ""
      for index_2 in range(len(eumjeol_sequence)):
          if(index_2 == 0):
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          if(correct_labels[index_2] == "B"):
              correct_sentence += " "
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):
              predict_sentence += " "
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

predict = crf.predict(test_x)

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))
print()

print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], predict[:10])

Accuracy score : 0.9240051256496049

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 특히 애플은 리튬이온 배터리를 교체하기만 해도 아이폰 성능이 70% 가량 복구될 수 있단 사실을 알리지 않은 점을 꼬집었다.
출력 문장 : 특히 애플은 리튬이 온배터리를 교체하기 만해도 아이폰 성능이 70% 가량 복구될 수 있단 사실을 알리지 않은 점을 꼬집었다.

정답 문장 : 이때 무인항공기가 촬영한 열 화상 (이름)가 방 어디에 누가 있는 지를 알려준다.”.
출력 문장 : 이 때무인 항공기가 촬영한 열화상 (이름)가 방어디에 누가 있는 지를 알려준다.”.

정답 문장 : 그는 그 (이름) “업그레이드하는 추가 이점이 크지 않기 때문”이라고 설명했다..
출력 문장 : 그는 그 (이름) “업그레이드하는 추가 이점이 크지 않기 때문”이라고 설명했다..

정답 문장 : 다만 원본 음원이 16비트이거나 24비트라 해도 처리 비트 수가 늘어나면 다이나믹 레인지가 증가하고 해상도가 높아지므로 음질 향상에는 기여할 것이라 본다.".
출력 문장 : 다만 원본 음원이 16비트이 거나 24비트라해도 처리비트수가 늘어나면다이 나믹레인지가 증가하고 해상도가 높아지므로 음질 향상에는 기여할 것이라본다.".

정답 문장 : #문제는 애플이다.
출력 문장 : #문제는 애플이다.

정답 문장 : 두 기업은 이같은 전략으로 흑자 전환과 수익 확대를 달성하겠다고 밝혔다.
출력 문장 : 두기업은 이 같은 전략으로 흑자 전환과 수익 확대를 달성하겠다고 밝혔다.

정답 문장 : 또 애플이 고의로 다른 사람의 자산에 개입해서 가치를 떨어뜨린 사실을 숨긴 점 역시 문제 삼았다..
출력 문장 : 또 애플이고 의로 다른 사람의 자산에 개입해서 가치를 떨어뜨린 사실을 숨긴 점역 시문제삼았다..

정답 문장 : 플랫폼 보안에선 ▲설정값 및 실행코드 무결성 검증 ▲안전한 업데이트 ▲감사기록을 요구했다.
출력 문장 : 플랫폼 보안에선 ▲설정값 및 실행코드무 결성 검증 ▲안전 